In [ ]:
import cv2
import numpy as np
#import bimpy
import time
from PIL import ImageFont, ImageDraw, Image
from find_circles import find_circles
from threading import Thread
from sklearn.metrics import r2_score
from Stack import *

In [ ]:
def distf(a,b):
    return math.sqrt(math.pow(a[0]-b[0],2)+math.pow(a[1]-b[1],2))

def move_to_storage():
    pass

def robot_turn(x,y):
    time.sleep(20)
    
def clean_place(x,y):
    pass

def test(m):
    print(m)
    
def clean_robot_plate(x,y):
    pass

def robot():
    global robot_work
    global turning_cutlets
    
    while True:
        if not turning_cutlets.is_empty():
            robot_work = True
            k = turning_cutlets.pop()
            k['turning'] = True
            x,y,r = k['pos']
            turn_cutlet(k)
            k['turning'] = False
            clean_place(x,y)
            robot_work = False
    

def add_cutlet(x,y,r):
    global table
    global start_time
    global order_number
    
    kotleta = {
    "pos": (x,y,r),
    "order": order_number,
    "fry":[
        [0,start_time],
    ],
    "status": 0.01,
    "turning":False,
    "r2score":0
   
    }
        
    table = np.append(table, kotleta)
    order_number += 1
    
    
    
def turn_cutlet(katleta):
    global start_time
    fry = katleta['fry']
    x,y,r = katleta['pos']
    if len(fry)<4:
        if fry[-1][0] == 1:
            fry.append([0,start_time])
        else:
            fry.append([1,start_time])
           
        robot_turn(x,y)
        
        circles = find_circles(frame) 
        for x,y,r in circles:
            found = False
            for k in table:
                
                x_t,y_t,r_t = k['pos']
                
                if distf((x,y),(x_t+r*2,y_t))<150:
                    k['pos'] = (x,y,r)
                    found = True
                    break
            if not found:
                add_cutlet(x,y,r)                
    
    else:
        move_to_storage()

In [ ]:
cap = cv2.VideoCapture("http://10.228.58.192:8080/video")#"http://192.168.1.65:8080/video" 10.228.58.192:8080

fourcc = cv2.VideoWriter_fourcc(*'MP4V')    
ret, frame = cap.read() 
v_out = None

turning_cutlets = Stack()
order_number = 1000
start_time = 0.0
fry_time = 20.0
all_time = fry_time*4
robot_work = False


thread = Thread(target=robot)
thread.start()


#инициализация bimpy
# ctx = bimpy.Context()
# ctx.init(1270, 720, "Cutlet")
#шрифт для PIL
font = ImageFont.truetype("arial.ttf", 32)

#создаем стол
table = np.array([])

#первый поиск
circles = find_circles(frame) 

for x, y, r in circles:
    add_cutlet(x,y,r)
                

while True:
    #получаем кадр, преобразовываем в PIL 
    key = cv2.waitKey(1)
    ret, frame = cap.read()
    h,w = frame.shape[:2]
    pil_im = Image.fromarray(frame)
    draw = ImageDraw.Draw(pil_im)
   
    #создаем фрейм bimby 
    #ctx.new_frame()
    #if bimpy.begin("Cutlet", flags=(bimpy.WindowFlags.AlwaysAutoResize | bimpy.WindowFlags.NoTitleBar)): 
        #выводим готовящиеся котлеты на столе
        
    for kotleta in table:

        if kotleta['status']<1 and not kotleta['turning']:

            status = kotleta['status']
            x,y,r = kotleta['pos']
            fry = kotleta['fry']
            order = "Заказ №" + str(kotleta['order'])
            #score = kotleta['r2score']

            indent = 0
            turn_text_size_h=0

            draw.rectangle((x-r+5, y + r - len(fry)*30, x+r-5, y+r+10), outline="white", fill="white")
            for f in fry:
                turn_text_size_w=0
                time_text = " {}:{} ".format(int(f[1]/60), int(f[1]%60) if f[1]%60>9 else '0'+ str(int(f[1]%60)))
                turn_text_size_w += draw.textsize(time_text, font=font)[0] + draw.textsize(order, font=font)[0]
                turn_text_size_h += draw.textsize(order, font=font)[1]

                side = "Сторона А " if f[0] == 0 else "Сторона Б "

                draw.text((x-draw.textsize(side, font=font)[0]/2-30,y+r-30-indent), side + time_text,(0,0,0), font=font)
                indent += 30


            #отрисовываем
            draw.rectangle((x-r-10, y-r-10, x+r+10, y+r+10), outline="green", width=5) #котлета   
            draw.rectangle((x-r-10, y+r+15, x+r+10, y+r+55), outline="green", width=5) #статус (пустой)
            draw.rectangle((x-r-10, y+r+15, x-r+(status*r*2), y+r+55), outline="green", fill="green") #статус 

            #draw.text((x-15, y+r+20), "{}:{} ".format(int(timer/60), int(t) if t>9 else '0'+ str(int(t)) ), font=font)   
            draw.rectangle((x-r-10, y-r-60, x+r+10, y-r-15), outline="green", width=5) #область заказа
            #draw.rectangle((x+r+15, y-r-10, x+r+turn_text_size_w+10, y-r+turn_text_size_h+10), outline="green", width=5) #переворачивания 
            draw.text((x-r,y-r-60), order, font=font) #заказ
            kotleta['status']+= 1/all_time 
            fry_size = len(fry)

            if (fry_size==4 and fry[-1][1]>=fry_time):
                top = draw.textsize(time_text, font=font)[1]*5
                draw.text((x+r+30, y-r-10+top), "Котлета готова!" , font=font)       
            else:
                f[1]+=1

            if fry[-1][1]>=fry_time:
                turning_cutlets.push(kotleta)

            if not robot_work:

                circles = find_circles(frame) 

                for x,y,r in circles:
                    if 130<r<160:
                        found = False
                        for k in table:
                            x_t,y_t,r_t = k['pos']

                            if distf((x,y),(x_t,y_t))<150:
                                score = r2_score((x,y,r),(x_t,y_t,r_t))
                                k['r2score'] = f" {score:.3f}" 

                                if score > 0.98:
                                    found = True
                                    break

                                k['pos'] = (x,y,r)
                                found = True
                                break
                        if not found:
                            add_cutlet(x,y,r) 
        if v_out is None:
            v_out = cv2.VideoWriter('interface_thread {}.mp4'.format(time.time()),fourcc, 5.0, (frame.shape[1],frame.shape[0]),3)
            
        v_out.write(np.array(pil_im))
        #изменяем размер и палитру
        #resized = cv2.resize(np.array(pil_im)[:, :, ::-1],(int(w/2),int(h/2)))
        #im = bimpy.Image(resized)
        #выводим на экран
        cv2.imshow("interface",np.array(pil_im))
          
#     bimpy.end()
#     ctx.render()
    del draw
   
    time.sleep(1)
    
    if key == ord('q') or ret == False:
        break 

v_out.release()
#if isinstance(v_out, type(None)) == False: v_out.release()
cv2.destroyAllWindows()


In [ ]:
v_out.release()